# Data Cleaning
Linear Regression for Apartment Prices in Stockholm, Sweden during 2024

In [1]:
# General imports
import folium
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from sklearn.model_selection import train_test_split

## 1. Introduction

While taking the course ["Supervised Machine Learning: Regression and Classification" (DeepLearning.AI)](https://www.coursera.org/learn/machine-learning?specialization=machine-learning-introduction), I felt the urge to create my own project in linear regression to get a deeper understanding.
<br> So, this is my take on Linear Regression using Machine Learning while following the course material. 

### 1.1 About the Project

While taking the course "Supervised Machine Learning: Regression and Classification" (DeepLearning.AI), I felt the urge to create my own project in linear regression to get a deeper understanding.
<br> So, this is my take on Linear Regression using Machine Learning while following the course material.

This project will be about applying linear regression to prices for apartments in Stockholm, Sweden during 2024.
<br> The dataset consists of data manually retrieved from the site [Booli.se](https://www.booli.se)
<br> The search parameters used were "Midsommarkransen", "Apartments only", "End prices from 2024-01-01". The data ends at 2023-09-12.
<br> I eliminated rows lacking important data and I also added data when it was missing (derived from other real estate ads from the same address). 
<br> This resulted in me having a dataset with 475 rows.
<br>
<br> The idea is to focus on a limited dataset and examine the behaviour of Linear Regressional modeling to look at parameter sensitivity, over-/underfitting, effect of regularization etc.
<br> There are a number of features I want to create which will be done under feature selection.

### 1.2 Highlights of Tasks in this Notebook

- Determining initial list of features
- Describing how the data was acquired
- Using Google Cloud API to access coordinates of apartments
- Creating of map of districts with Folium and coordinates
- Assigning districts to apartments
- General cleaning of data
- Creating a test set early on, based on Stratified Sampling   

## 2. Features
I am interested in looking at the following features:
- `Living area` (m<sup>2</sup>)
- `Side area` (m<sup>2</sup>) (areas with slanted ceiling)
- `Total area` (m<sup>2</sup>) (Living and Side area)
- `District`
- `Number of rooms`
- `Monthly fee` (sek=Swedish krona) (Payed to the the condominium association - maintenece)
- `Floor`
- `Year of building` or `Age of building`
- `Elevator` (Yes/No)
- `Balcony` (Yes/No)
- `Patio` (Yes/No)
- `Fireplace` (Yes/No)
- `Date of sale` (not sure if it will be used)
- `Days For Sale` (not sure if it will be used)
- `Coordinates` (will be used in Feature Selection/Creation to make the shortest distance to `Subway Station` and `Grocery Store` (other features may be `Café`, `Bakery`, `Gym`)

I also have final sales `Price` for each apartment.
<br> The coordinates were collected via an `API request` (Geocoding API) using Google Cloud using a python script.

## 3. Acquisition of Dataset

As mentioned previously, the dataset was made by entering data into an Excel sheet and later on converted to a csv-file.
<br> I did look at the possibility of webscraping using [BeatifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), but according to their [robots.txt-file](https://www.booli.se/robots.txt) I thought I saw that they didn't allow web scraping. Looking at it now a bit closer, it seems as they don't hinder using web scraping.
<br> I also looked at their API, but it was closed. So I decided to just entered the data myself into my own Excel file. 
<br>
<br> This means that there may be some errors and I know there are some missing information. We'll see how much cleaning we can do. 
<br> Some of the missing information has to do with different real estate agents entering data with various degrees of granualarity. 

## 4. A First Look

In [2]:
# Path to the raw CSV file from GitHub
path = 'https://raw.githubusercontent.com/gustmic/Project-LinReg-Apartments/main/data/dataset_apartments.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')

In [3]:
df.head()

,Address,District,Price,Living area,Side area,Rooms,Monthly Fee,Floor,Year of Building,Elevator,Balcony,Patio,Fireplace,Date of sale,Days for sale
0,Adventsvägen 1,NaN,6800000,114,0,4,5767,1,2017.0,1,0.0,0.0,0.0,2024-09-04,15.0
1,Adventsvägen 3,NaN,3 195 000,52,NaN,2,4355,1,2017.0,0,1.0,NaN,NaN,2024-02-28,102.0
2,Adventsvägen 3,NaN,2 675 000,31,NaN,"1,5",2589,3,2017.0,0,0.0,NaN,NaN,2024-03-15,28.0
3,Adventsvägen 3,NaN,2 675 000,31,NaN,1,2589,4,2017.0,1,1.0,NaN,NaN,2024-03-26,65.0
4,Adventsvägen 3,Midsommarkransen,3 820 000,48,NaN,2,4019,3,2017.0,1,1.0,NaN,NaN,2024-04-20,15.0


As we can see, we have some work to do:
- `Districts`. Set up uniform `Districts`. As of now, these are set up by individual real estate agents. I will indicate them on a map and order them into district according to their coordinates.
- `Price`. Uniform format.
- `Living Area`. Make sure the comma is converted to a decimal.
- `Side Area`. replace NaN with 0.
- `Rooms`. Make sure the comma is converted to a decimal.
- `Patio`. Make it datatype INT.
- `Fireplace`. Make it datatype INT.
- `Date of Sale`. Convert to DateTime.
- `Coordinates`. Split into Long and Lat. 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          128 non-null    object 
 2   Price             475 non-null    object 
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
dtypes: float64(5), int64(2), object(8)
memory usage: 55.8+ KB


## 5. Cleaning the Dataset

### 5.1 Address

There is nothing to change here.

### 5.2 Coordinates

Let's create `Address` into `Latitude` and `Longitude`.
<br> We do this by calling Google Maps Geocoding API.

In [5]:
# Define your API key
API_KEY = '*********************' # Enter your API_KEY here

# Prepare columns for latitude and longitude
df['Latitude'] = None
df['Longitude'] = None

# Geocoding function
def get_lat_lon(address):
    # Append ", Stockholm" to the partial address
    full_address = f"{address}, Stockholm"
    
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': full_address, 'key': API_KEY}
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        if data['status'] == 'OK':
            results = data['results']
            if results:
                # Choose the first result as the best match
                location = results[0]['geometry']['location']
                return location['lat'], location['lng']
            else:
                print("No results found.")
        else:
            print(f"Geocoding API Error: {data['status']}")
    else:
        print(f"HTTP Request Error: {response.status_code}")
    
    return None, None


# Process each address, skipping rows where 'Address' is NaN
for i, row in df.iterrows():
    address = row['Address']
    
    if pd.notna(address):  # Only proceed if the address is not NaN
        lat, lon = get_lat_lon(address)
        df.at[i, 'Latitude'] = lat
        df.at[i, 'Longitude'] = lon
    else:
        print(f"Skipping row {i} due to missing address.")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          128 non-null    object 
 2   Price             475 non-null    object 
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    object 
 16  Longitude         475 non-null    object 
dt

In [7]:
df.head()

,Address,District,Price,Living area,Side area,Rooms,Monthly Fee,Floor,Year of Building,Elevator,Balcony,Patio,Fireplace,Date of sale,Days for sale,Latitude,Longitude
0,Adventsvägen 1,NaN,6800000,114,0,4,5767,1,2017.0,1,0.0,0.0,0.0,2024-09-04,15.0,59.301007,18.007276
1,Adventsvägen 3,NaN,3 195 000,52,NaN,2,4355,1,2017.0,0,1.0,NaN,NaN,2024-02-28,102.0,59.30089,18.007191
2,Adventsvägen 3,NaN,2 675 000,31,NaN,"1,5",2589,3,2017.0,0,0.0,NaN,NaN,2024-03-15,28.0,59.30089,18.007191
3,Adventsvägen 3,NaN,2 675 000,31,NaN,1,2589,4,2017.0,1,1.0,NaN,NaN,2024-03-26,65.0,59.30089,18.007191
4,Adventsvägen 3,Midsommarkransen,3 820 000,48,NaN,2,4019,3,2017.0,1,1.0,NaN,NaN,2024-04-20,15.0,59.30089,18.007191


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          128 non-null    object 
 2   Price             475 non-null    object 
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    object 
 16  Longitude         475 non-null    object 
dt

Before we move on to `District` we must make sure that all values in `Latitude` and `Longitude` are numerical.

In [9]:
# Convert 'Latitude' and 'Longitude' to numeric (float), coercing invalid values to NaN
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Verify the changes
print(df[['Latitude', 'Longitude']].dtypes)

Latitude     float64
Longitude    float64
dtype: object


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          128 non-null    object 
 2   Price             475 non-null    object 
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

Looks good.
<br> Let's make sure that we don't have errors in the coordinates, i.e. wierd coordinates or non-existing values

In [11]:
# Check for non-numerical values in 'Latitude' and 'Longitude'
df['Latitude_numeric'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude_numeric'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Check for any NaN values in the numeric columns
non_numerical_latitude = df['Latitude_numeric'].isna().any()
non_numerical_longitude = df['Longitude_numeric'].isna().any()

if non_numerical_latitude:
    print("There are non-numerical values in the 'Latitude' column.")
else:
    print("All values in the 'Latitude' column are numerical.")

if non_numerical_longitude:
    print("There are non-numerical values in the 'Longitude' column.")
else:
    print("All values in the 'Longitude' column are numerical.")

# Optionally, drop the temporary columns if not needed
df.drop(columns=['Latitude_numeric', 'Longitude_numeric'], inplace=True)

All values in the 'Latitude' column are numerical.
All values in the 'Longitude' column are numerical.


Looks good.

In [12]:
df.describe()

,Monthly Fee,Year of Building,Elevator,Balcony,Patio,Fireplace,Days for sale,Latitude,Longitude
count,475.000000,449.000000,475.000000,474.000000,91.000000,87.000000,473.000000,475.000000,475.000000
mean,3679.183158,1971.322940,0.454737,0.502110,0.241758,0.241379,35.158562,59.299371,18.002055
std,1155.661119,38.801814,0.498472,0.500524,0.430521,0.430400,61.285161,0.003017,0.006414
min,1707.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.290327,17.988149
25%,2845.500000,1940.000000,0.000000,0.000000,0.000000,0.000000,10.000000,59.297593,17.997286
50%,3411.000000,1944.000000,0.000000,1.000000,0.000000,0.000000,14.000000,59.299873,18.002455
75%,4285.000000,2013.000000,1.000000,1.000000,0.000000,0.000000,32.000000,59.301604,18.006189
max,8284.000000,2024.000000,1.000000,1.000000,1.000000,1.000000,470.000000,59.304204,18.015493


**Comment**
<br> The `Latitude` and `Longitude` both look good in terms of value ranges.

### 5.3 District

We now only have to fix the 'District' column. 
<bt> There already exists some values there, but these are entered from what the real estate agency deemed the object to belong to. 
<br> In the area "Midsommarkransen", an inner suburb to Stockholm, there aren't actually official districts. 
<br> However, having first hand experience from the area, I know that some areas of Midsommarkransen are valued higher in peoples opinions. 
<br> So, I thought it may be interesting to add it as a feature.
<br> We will use informal and historical names that are used by people living in the area.
<br>
<br> We will define areas within certain coordinates, as polygons, assign names/numbers to these districts, and finally assign the apartments to these districts.
<br> At the very end, I will assign addresses with houses that have a 'Year of Building'<= 1920 to 'District' == 'Gamla Midsommarkransen' (trans. 'Old Midsommarkransen').
<br> The background to this is that we have apartments/buildings that make up the original apartments/buildings that started the area 'Midsommarkransen'.

In [13]:
# Define polygons for each district (example coordinates)
district_polygons = {
    1: Polygon([(18.004156569121477, 59.30090784434475), (18.006542466249208, 59.303536039023335), (18.003308108384427, 59.30354906550476), (18.001572910476696, 59.302513444665585), (18.000360823702913, 59.30275444100907), (17.997987685382537, 59.30229524379241), (17.995702455812182, 59.30163790886046), (17.995002124276585, 59.30129540998905), (17.995270059244486, 59.30020109563427), (17.996424731383566, 59.29830876068701), (17.998115273462787, 59.29864424254005), (18.00113911099409, 59.30011641507348), (18.00307844983214, 59.30060495382102), (18.00324431434554, 59.30097949547381)]),
    2: Polygon([(17.99957193725194, 59.29416724083719), (17.996728795778786, 59.291986739767346), (17.9917828024887, 59.29028826274625), (17.986107248298183, 59.28985541149427), (17.985324043074623, 59.29033757457149), (17.987426894710957, 59.291937430321624), (17.98890202548726, 59.29708600252817), (17.99180953973953, 59.29726677605906), (17.99581139575144, 59.295941081186214), (17.99611180312806, 59.29549186707748)]),
    3: Polygon([(17.98890202548726, 59.29708600252817), (17.98749651998394, 59.29839519151933), (17.987754012149693, 59.2997646113716), (17.99156273435273, 59.30061906464913), (17.995002124276585, 59.30129540998905), (17.995270059244486, 59.30020109563427), (17.996424731383566, 59.29830876068701), (17.998115273462787, 59.29864424254005), (18.002811115370527, 59.29869523342455), (18.002667485926423, 59.29851783894495), (18.005206301469677, 59.29770552181718), (17.999947494068802, 59.2941365592012), (17.99957193725194, 59.29416724083719), (17.99611180312806, 59.29549186707748), (17.99581139575144, 59.295941081186214), (17.99180953973953, 59.29726677605906)]),
    4: Polygon([(17.988837635873434, 59.30279356694878), (17.993429539256592, 59.304113525613566), (18.001714335838713, 59.304700577610674), (18.009213791922114, 59.305335877510586), (18.016402111974603, 59.305116809915), (18.019341813172957, 59.30358329727191), (18.014481650757833, 59.30037365065614), (18.01130624585736, 59.299551390719465), (18.005206301469677, 59.29770552181718), (18.002667485926423, 59.29851783894495), (18.002811115370527, 59.29869523342455), (17.998115273462787, 59.29864424254005), (18.00113911099409, 59.30011641507348), (18.00307844983214, 59.30060495382102), (18.00324431434554, 59.30097949547381), (18.004156569121477, 59.30090784434475), (18.006542466249208, 59.303536039023335), (18.003308108384427, 59.30354906550476), (18.001572910476696, 59.302513444665585), (18.000360823702913, 59.30275444100907), (17.997987685382537, 59.30229524379241), (17.995702455812182, 59.30163790886046), (17.995002124276585, 59.30129540998905), (17.99156273435273, 59.30061906464913)])
}

# Define colors and names for each district
district_info = {
    1: {"name": "LM-Staden", "color": "#228B22"},  # Forest Green
    2: {"name": "Hökmossen", "color": "#8B4513"},  # Saddle Brown
    3: {"name": "Telefonplan", "color": "#4169E1"},  # Royal Blue
    4: {"name": "Midsommarkransen", "color": "#DC143C"}  # Crimson
}

def style_function(feature):
    district_id = feature['properties']['district_id']
    return {
        'fillColor': district_info[district_id]['color'],
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.2
    }

# Initialize a folium map centered around the general area of the districts
m = folium.Map(location=[59.30000476188227, 17.99730815309434], zoom_start=15)

# Add polygons to the map with predetermined colors
for district_id, polygon in district_polygons.items():
    geojson = {
        "type": "Feature",
        "geometry": polygon.__geo_interface__,
        "properties": {
            "district_id": district_id,
            "name": district_info[district_id]['name']
        }
    }
    
    folium.GeoJson(
        data=geojson,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['District'])
    ).add_to(m)

# Create a large bounding box covering the entire earth
bounding_box = Polygon([
    (-180, -90), (180, -90),
    (180, 90), (-180, 90),
    (-180, -90)
])

# Combine all district polygons into one MultiPolygon
combined_district_polygons = unary_union(list(district_polygons.values()))

# Subtract the combined district polygons from the bounding box
grey_area = bounding_box.difference(combined_district_polygons)

# Add the grey area to the map
folium.GeoJson(
    data=grey_area.__geo_interface__,
    style_function=lambda x: {
        'fillColor': 'grey',
        'color': 'grey',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)

# Add black circle markers for each row's 'Latitude' and 'Longitude'
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=2,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.8
    ).add_to(m)

# Add a legend including the 'Object' circles
legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 140px; 
    border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
    ">&nbsp; <b>District Legend</b> <br>
    '''
for district_id, info in district_info.items():
    legend_html += f'''
    &nbsp; <i class="fa fa-square fa-1x"
              style="color:{info['color']};"></i>&nbsp; {info['name']} <br>
    '''
legend_html += '''
    &nbsp; <i class="fa fa-circle fa-1x"
              style="color:black;"></i>&nbsp; Apartment Location <br>
    '''
legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
m.save("map_with_districts_and_objects.html")

# Display map
m

We now assign the districts to the apartments.

In [14]:
# Function to determine which district a coordinate falls into
district_polygons = {
    'LM-Staden': Polygon([(59.30090784434475, 18.004156569121477), (59.303536039023335, 18.006542466249208), (59.30354906550476, 18.003308108384427), (59.302513444665585, 18.001572910476696), (59.30275444100907, 18.000360823702913), (59.30229524379241, 17.997987685382537), (59.30163790886046, 17.995702455812182), (59.30129540998905, 17.995002124276585), (59.30020109563427, 17.995270059244486), (59.29830876068701, 17.996424731383566), (59.29864424254005, 17.998115273462787), (59.30011641507348, 18.00113911099409), (59.30060495382102, 18.00307844983214), (59.30097949547381, 18.00324431434554)]),
    'Hökmossen': Polygon([(59.29416724083719, 17.99957193725194), (59.291986739767346, 17.996728795778786), (59.29028826274625, 17.9917828024887), (59.28985541149427, 17.986107248298183), (59.29033757457149, 17.985324043074623), (59.291937430321624, 17.987426894710957), (59.29708600252817, 17.98890202548726), (59.29726677605906, 17.99180953973953), (59.295941081186214, 17.99581139575144), (59.29549186707748, 17.99611180312806)]),
    'Telefonplan': Polygon([(59.29708600252817, 17.98890202548726), (59.29839519151933, 17.98749651998394), (59.2997646113716, 17.987754012149693), (59.30061906464913, 17.99156273435273), (59.30129540998905, 17.995002124276585), (59.30020109563427, 17.995270059244486), (59.29830876068701, 17.996424731383566), (59.29864424254005, 17.998115273462787), (59.29869523342455, 18.002811115370527), (59.29851783894495, 18.002667485926423), (59.29770552181718, 18.005206301469677), (59.2941365592012, 17.999947494068802), (59.29416724083719, 17.99957193725194), (59.29549186707748, 17.99611180312806), (59.295941081186214, 17.99581139575144), (59.29726677605906, 17.99180953973953)]),
    'Midsommarkransen': Polygon([(59.30279356694878, 17.988837635873434), (59.304113525613566, 17.993429539256592), (59.304700577610674, 18.001714335838713), (59.305335877510586, 18.009213791922114), (59.305116809915, 18.016402111974603), (59.30358329727191, 18.019341813172957), (59.30037365065614, 18.014481650757833), (59.299551390719465, 18.01130624585736), (59.29770552181718, 18.005206301469677), (59.29851783894495, 18.002667485926423), (59.29869523342455, 18.002811115370527), (59.29864424254005, 17.998115273462787), (59.30011641507348, 18.00113911099409), (59.30060495382102, 18.00307844983214), (59.30097949547381, 18.00324431434554), (59.30090784434475, 18.004156569121477), (59.303536039023335, 18.006542466249208), (59.30354906550476, 18.003308108384427), (59.302513444665585, 18.001572910476696), (59.30275444100907, 18.000360823702913), (59.30229524379241, 17.997987685382537), (59.30163790886046, 17.995702455812182), (59.30129540998905, 17.995002124276585), (59.30061906464913, 17.99156273435273)])
}

def assign_district(row):
    point = Point(row['Latitude'], row['Longitude'])
    for district_id, polygon in district_polygons.items():
        if polygon.contains(point):
            return district_id
    return None  # If not in any district

# Apply function to each row in the DataFrame
df['District'] = df.apply(assign_district, axis=1)

In [15]:
df['District'].value_counts().sort_index(ascending=True)

District
Hökmossen            45
LM-Staden            73
Midsommarkransen    219
Telefonplan         138
Name: count, dtype: int64

**Comment**
<br> We will finally create the `District` - 'Gamla Midsommarkransen', for all apartments where `Year of Building` <= 1920, but only for `District` is Midsommarkransen.

In [16]:
df.loc[(df['District'] == 'Midsommarkransen') & (df['Year of Building'] <= 1920), 'District'] = 'Gamla Midsommarkransen'

In [17]:
df['District'].value_counts().sort_index(ascending=True)

District
Gamla Midsommarkransen     23
Hökmossen                  45
LM-Staden                  73
Midsommarkransen          196
Telefonplan               138
Name: count, dtype: int64

### 5.4 Price

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    object 
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

In [19]:
# Checking if there are any non-numeric values

# Convert all values in the 'Price' column to strings
df['Price'] = df['Price'].astype(str)

# Identify non-numeric values in the 'Price' column
non_numeric_prices = df[~df['Price'].str.match(r'^\d+$')]

print("Non-numeric values in 'Price' column:")
print(non_numeric_prices['Price'])

Non-numeric values in 'Price' column:
1      3 195 000
2      2 675 000
3      2 675 000
4      3 820 000
5      2 830 000
         ...    
468    4 550 000
469    3 050 000
470    2 800 000
472    3 110 000
473    4 460 000
Name: Price, Length: 402, dtype: object


**Comment**
<br> We can see that 402 out of 476 values are non-numerical.
<br> Let's convert them to datatype Int.

In [20]:
# Function to clean and convert price strings to integers
def clean_and_convert_price(price):
    try:
        # Remove any non-numeric characters except decimal points
        cleaned_price = ''.join(filter(str.isdigit, price))
        return int(cleaned_price)
    except ValueError:
        return None  # Return None for values that cannot be converted

# Apply the function to the 'Price' column
df['Price'] = df['Price'].apply(clean_and_convert_price)

# Replace NaN values with 0 if any non-convertible values were found
df['Price'] = df['Price'].fillna(0).astype(int)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    int32  
 3   Living area       475 non-null    object 
 4   Side area         76 non-null     object 
 5   Rooms             473 non-null    object 
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    object 
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

In [22]:
df.head()

,Address,District,Price,Living area,Side area,Rooms,Monthly Fee,Floor,Year of Building,Elevator,Balcony,Patio,Fireplace,Date of sale,Days for sale,Latitude,Longitude
0,Adventsvägen 1,Midsommarkransen,6800000,114,0,4,5767,1,2017.0,1,0.0,0.0,0.0,2024-09-04,15.0,59.301007,18.007276
1,Adventsvägen 3,Midsommarkransen,3195000,52,NaN,2,4355,1,2017.0,0,1.0,NaN,NaN,2024-02-28,102.0,59.300890,18.007191
2,Adventsvägen 3,Midsommarkransen,2675000,31,NaN,"1,5",2589,3,2017.0,0,0.0,NaN,NaN,2024-03-15,28.0,59.300890,18.007191
3,Adventsvägen 3,Midsommarkransen,2675000,31,NaN,1,2589,4,2017.0,1,1.0,NaN,NaN,2024-03-26,65.0,59.300890,18.007191
4,Adventsvägen 3,Midsommarkransen,3820000,48,NaN,2,4019,3,2017.0,1,1.0,NaN,NaN,2024-04-20,15.0,59.300890,18.007191


Looks good.

### 5.5 Living area, Side Area, Rooms and Floor

I want to replace commas with decimal points.

In [23]:
columns = ['Living area', 'Side area', 'Rooms', 'Floor']

for col in columns:
    # Step 1: Convert 'Rooms' column to string (if it's not already)
    df[col] = df[col].astype(str)

    # Step 2: Replace commas with decimal points
    df[col] = df[col].str.replace(',', '.', regex=False)

    # Handle "nan" and other non-numeric values
    # Convert non-numeric strings to NaN
    df[col] = df[col].replace('nan', np.nan)

    # Step 3: Convert 'Rooms' column to numeric (float), coercing errors
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    int32  
 3   Living area       475 non-null    float64
 4   Side area         76 non-null     float64
 5   Rooms             473 non-null    float64
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    float64
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

**Comment**
<br> Looks good. 
<br> `District`, `Price`, `Living Area` and `Rooms` are all done.
<br> 
<br> Let's fill NaN-values of `Side area` as 0.
<br> We'll leave `Rooms` "as is", while we figure out if we want to use it at all.
<br> We lack some values in the feature `Floor`, and we'll leave this one in a similar way to `Rooms`. 

In [25]:
# Fill NaN values with 0 in the 'Side Area' column
df['Side area'] = df['Side area'].fillna(0)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    int32  
 3   Living area       475 non-null    float64
 4   Side area         475 non-null    float64
 5   Rooms             473 non-null    float64
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    float64
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           474 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

### 5.6 Monthly Fee, Year of Building, Elevator, Balcony

No need to change `Monthly Fee` or `Elevator`.
<br> `Year of Building` lacks some values, but similar to `Rooms` we will analyze it first to see if we'll use it at all.
<br> We will fill NaN-value for `Balcony` as 0, assuming a non-existant value is equal to a lack of balcony.
<br>
<br> Note that we do not want to delete rows at this stage since they may contain valuable information in other features.

In [27]:
# Fill NaN values with 0 in the 'Balcony' column
df['Balcony'] = df['Balcony'].fillna(0)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    int32  
 3   Living area       475 non-null    float64
 4   Side area         475 non-null    float64
 5   Rooms             473 non-null    float64
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    float64
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           475 non-null    float64
 11  Patio             91 non-null     float64
 12  Fireplace         87 non-null     float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

### 5.7 Patio and Fireplace

Similar to `Side area`, let's fill out NaN-values to 0.

In [29]:
df[['Patio', 'Fireplace']] = df[['Patio', 'Fireplace']].fillna(0)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           475 non-null    object 
 1   District          475 non-null    object 
 2   Price             475 non-null    int32  
 3   Living area       475 non-null    float64
 4   Side area         475 non-null    float64
 5   Rooms             473 non-null    float64
 6   Monthly Fee       475 non-null    int64  
 7   Floor             440 non-null    float64
 8   Year of Building  449 non-null    float64
 9   Elevator          475 non-null    int64  
 10  Balcony           475 non-null    float64
 11  Patio             475 non-null    float64
 12  Fireplace         475 non-null    float64
 13  Date of sale      475 non-null    object 
 14  Days for sale     473 non-null    float64
 15  Latitude          475 non-null    float64
 16  Longitude         475 non-null    float64
dt

### 5.8 Date of sale

Let's convert to DateTime

In [31]:
df['Date of sale'] = pd.to_datetime(df['Date of sale'], format='%Y-%m-%d')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Address           475 non-null    object        
 1   District          475 non-null    object        
 2   Price             475 non-null    int32         
 3   Living area       475 non-null    float64       
 4   Side area         475 non-null    float64       
 5   Rooms             473 non-null    float64       
 6   Monthly Fee       475 non-null    int64         
 7   Floor             440 non-null    float64       
 8   Year of Building  449 non-null    float64       
 9   Elevator          475 non-null    int64         
 10  Balcony           475 non-null    float64       
 11  Patio             475 non-null    float64       
 12  Fireplace         475 non-null    float64       
 13  Date of sale      475 non-null    datetime64[ns]
 14  Days for sale     473 non-

### 5.9 Days for sale

We can see that we have two null values for `Days for sale`. 
<br> Let's examine them.

In [33]:
null_rows = df[df['Days for sale'].isnull()]

In [34]:
null_rows

,Address,District,Price,Living area,Side area,Rooms,Monthly Fee,Floor,Year of Building,Elevator,Balcony,Patio,Fireplace,Date of sale,Days for sale,Latitude,Longitude
165,Midsommarvägen 52,Midsommarkransen,3500000,48.0,0.0,2.0,4241,1.0,1944.0,0,0.0,0.0,0.0,2024-02-01,NaN,59.300435,18.007439
455,Vattenledningsvägen 42B,Gamla Midsommarkransen,5300000,59.0,0.0,2.0,3586,2.0,1910.0,0,0.0,0.0,0.0,2024-02-22,NaN,59.302709,18.009099


**Comment**
<br> Let's convert these NaN-values to 0.
<br> We don't know for how long these objects have been for sale, but let's set them to 0 for now.

In [35]:
df['Days for sale'] = df['Days for sale'].fillna(0)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Address           475 non-null    object        
 1   District          475 non-null    object        
 2   Price             475 non-null    int32         
 3   Living area       475 non-null    float64       
 4   Side area         475 non-null    float64       
 5   Rooms             473 non-null    float64       
 6   Monthly Fee       475 non-null    int64         
 7   Floor             440 non-null    float64       
 8   Year of Building  449 non-null    float64       
 9   Elevator          475 non-null    int64         
 10  Balcony           475 non-null    float64       
 11  Patio             475 non-null    float64       
 12  Fireplace         475 non-null    float64       
 13  Date of sale      475 non-null    datetime64[ns]
 14  Days for sale     475 non-

### 5.10 Convert to Datatype Int

In [37]:
columns_to_convert = ['Monthly Fee', 'Elevator', 'Balcony', 'Patio', 'Fireplace', 'Days for sale'] # Note that we do not include 'Year of Building' due to NaN-values
df[columns_to_convert] = df[columns_to_convert].astype(int)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Address           475 non-null    object        
 1   District          475 non-null    object        
 2   Price             475 non-null    int32         
 3   Living area       475 non-null    float64       
 4   Side area         475 non-null    float64       
 5   Rooms             473 non-null    float64       
 6   Monthly Fee       475 non-null    int32         
 7   Floor             440 non-null    float64       
 8   Year of Building  449 non-null    float64       
 9   Elevator          475 non-null    int32         
 10  Balcony           475 non-null    int32         
 11  Patio             475 non-null    int32         
 12  Fireplace         475 non-null    int32         
 13  Date of sale      475 non-null    datetime64[ns]
 14  Days for sale     475 non-

**Comment**
<br> I am happy with the cleaning.
<br> As a last step, let's make sure we don't have any non-numeric values hidden in the columns (besides `Address` and `District` as well as `Rooms`, `Floor`, and `Year of Building` that all have some NaN-values).

### 5.11 Checking for non-numeric values

In [39]:
# Function to check for non-numeric values in columns
def check_non_numeric(df):
    for col in df.columns:
        try:
            # Convert to numeric, coercing errors to NaN
            numeric_values = pd.to_numeric(df[col], errors='coerce')
            # Check for non-numeric values (NaN values)
            non_numeric_mask = numeric_values.isna()

            if non_numeric_mask.any():
                print(f"The column '{col}' contains non-numeric values.")
        except Exception as e:
            print(f"An error occurred with column '{col}': {e}")

# Run the check
check_non_numeric(df)


The column 'Address' contains non-numeric values.
The column 'District' contains non-numeric values.
The column 'Rooms' contains non-numeric values.
The column 'Floor' contains non-numeric values.
The column 'Year of Building' contains non-numeric values.


**Comment**
<br> Note that we expect to see this result, as `Address` and `District` naturally are objects (string values).
<br> `Rooms`, `Floor` and `Year of Building` are missing values, and as such these missing values are NaN-values which are non-numerical.
<br> Thus, all is well.

In [40]:
df.head()

,Address,District,Price,Living area,Side area,Rooms,Monthly Fee,Floor,Year of Building,Elevator,Balcony,Patio,Fireplace,Date of sale,Days for sale,Latitude,Longitude
0,Adventsvägen 1,Midsommarkransen,6800000,114.0,0.0,4.0,5767,1.0,2017.0,1,0,0,0,2024-09-04,15,59.301007,18.007276
1,Adventsvägen 3,Midsommarkransen,3195000,52.0,0.0,2.0,4355,1.0,2017.0,0,1,0,0,2024-02-28,102,59.300890,18.007191
2,Adventsvägen 3,Midsommarkransen,2675000,31.0,0.0,1.5,2589,3.0,2017.0,0,0,0,0,2024-03-15,28,59.300890,18.007191
3,Adventsvägen 3,Midsommarkransen,2675000,31.0,0.0,1.0,2589,4.0,2017.0,1,1,0,0,2024-03-26,65,59.300890,18.007191
4,Adventsvägen 3,Midsommarkransen,3820000,48.0,0.0,2.0,4019,3.0,2017.0,1,1,0,0,2024-04-20,15,59.300890,18.007191


Looks like we are fine.

### 5.12 Renaming Columns

In [41]:
df.rename(columns={'Living area': 'Living Area', 'Side area': 'Side Area', 'Date of sale': 'Date of Sale', 'Days for sale': 'Days for Sale'}, inplace=True)

## 6. Creating a Test Set

### 6.1 Stratified Sampling

We will now create a Test Set to set aside before exploring the data.
<br> This to avoid *data snooping bias*.
<br>
<br> Since we have a rather small dataset, we will use a *stratified sampling* approach that will create a Test Set that is representative of the dataset as a whole.
<br> I think a good way of splitting it up is in regard to `District` since this should represent the dataset in a good way.

In [42]:
# Split the data into training and testing sets using stratification based on 'District'
train_df, test_df = train_test_split(df, stratify=df['District'], test_size=0.2, random_state=42)

# Display the shapes of the resulting DataFrames
print(f"Training set shape: {train_df.shape}")
print(f"Testing set shape: {test_df.shape}")

Training set shape: (380, 17)
Testing set shape: (95, 17)


### 6.2 Distribution Check

Let's take a look at the distribution of `Districts` of both the Training Set and the original Dataframe.

In [43]:
train_df['District'].value_counts() / len(train_df)

District
Midsommarkransen          0.413158
Telefonplan               0.292105
LM-Staden                 0.152632
Hökmossen                 0.094737
Gamla Midsommarkransen    0.047368
Name: count, dtype: float64

Compared to original distribution, df.

In [44]:
df['District'].value_counts() / len(df)

District
Midsommarkransen          0.412632
Telefonplan               0.290526
LM-Staden                 0.153684
Hökmossen                 0.094737
Gamla Midsommarkransen    0.048421
Name: count, dtype: float64

Looks consistent.

## 7. Saving the Datasets

Let's save this cleaned datasets (Training and Testing) to csv and then we'll move onto Feature Engineering in the next Notebook.

In [44]:
train_df.to_csv(r'...localpath...\cleaned_training_set_apartments.csv', index=False)
test_df.to_csv(r'...localpath...\cleaned_test_set_apartments.csv', index=False)

In [45]:
training_df = pd.read_csv(r'...localpath...\cleaned_training_set_apartments.csv')
training_df.shape

(380, 17)

Looks good.